<a href="https://colab.research.google.com/github/lansotto/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)
application_df.head()
#application_df.drop(application_df.columns[[0, 1]], axis=1, inplace=True)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
list_of_application_type_values = application_df['APPLICATION_TYPE'].value_counts().index.tolist()
list_of_application_type_values

['T3',
 'T4',
 'T6',
 'T5',
 'T19',
 'T8',
 'T7',
 'T10',
 'T9',
 'T13',
 'T12',
 'T2',
 'T25',
 'T14',
 'T29',
 'T15',
 'T17']

In [6]:
del list_of_application_type_values[:8]
application_types_to_replace = list_of_application_type_values

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#application_types_to_replace = ['T9','T13','T12',"T2","T25","T14","T29","T15","T17"]
application_types_to_replace = ["T15","T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
T25          3
T14          3
Other        3
T29          2
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [10]:
# Get list of columns from value_counts
list_of_classification_values = application_df['CLASSIFICATION'].value_counts().index.tolist()
list_of_classification_values

['C1000',
 'C2000',
 'C1200',
 'C3000',
 'C2100',
 'C7000',
 'C1700',
 'C4000',
 'C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [11]:
# creating more bins - by dropping fewer classification values to optimize model
# drop items from list_of_classification_values
del list_of_classification_values[:25]
classifications_to_replace = list_of_classification_values

In [12]:
classifications_to_replace

['C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [13]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
Other      143
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
Name: CLASSIFICATION, dtype: int64

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_df = pd.get_dummies(application_df)
categorical_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [15]:
# Generate our categorical variable lists
attrition_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
attrition_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [16]:
# Merge X  with application_df
df = application_df.merge(categorical_df,left_index = True, right_index=True)
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS_x,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT_x,IS_SUCCESSFUL_x,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,...,0,0,0,0,0,0,0,0,1,0
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,...,1,0,0,0,0,0,0,0,1,0
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,0,0,0,0,0,1,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,...,0,1,0,0,0,0,0,0,1,0
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,0,0,0,0,0,1,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,0,0,0,0,0,1,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0,...,0,0,0,0,0,0,0,0,1,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1,...,0,0,0,0,0,0,0,0,1,0


In [17]:
# Remove the original columns that have been replaced by numerical data
df_dropped_columns = df.drop(attrition_cat, 1)
df_dropped_columns

<ipython-input-17-24edfb2a080c>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_dropped_columns = df.drop(attrition_cat, 1)


,STATUS_x,ASK_AMT_x,IS_SUCCESSFUL_x,STATUS_y,ASK_AMT_y,IS_SUCCESSFUL_y,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,5000,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,1,108590,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,1,5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,1,6692,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,1,142590,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,1,5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,1,5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,1,5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,1,5000,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
column_headers = df_dropped_columns.columns.values.tolist()
print("The Column Header :", column_headers)

The Column Header : ['STATUS_x', 'ASK_AMT_x', 'IS_SUCCESSFUL_x', 'STATUS_y', 'ASK_AMT_y', 'IS_SUCCESSFUL_y', 'APPLICATION_TYPE_Other', 'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T12', 'APPLICATION_TYPE_T13', 'APPLICATION_TYPE_T14', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T2', 'APPLICATION_TYPE_T25', 'APPLICATION_TYPE_T29', 'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6', 'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8', 'APPLICATION_TYPE_T9', 'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent', 'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other', 'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200', 'CLASSIFICATION_C1230', 'CLASSIFICATION_C1240', 'CLASSIFICATION_C1270', 'CLASSIFICATION_C1280', 'CLASSIFICATION_C1300', 'CLASSIFICATION_C1400', 'CLASSIFICATION_C1500', 'CLASSIFICATION_C1700', 'CLASSIFICATION_C1800', 'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C2300', 'CLASSIFICATION_C270

In [19]:
df_dropped_columns

,STATUS_x,ASK_AMT_x,IS_SUCCESSFUL_x,STATUS_y,ASK_AMT_y,IS_SUCCESSFUL_y,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,5000,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,1,108590,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,1,5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,1,6692,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,1,142590,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,1,5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,1,5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,1,5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,1,5000,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [20]:
df_dropped_columns.nunique()

STATUS_x                       2
ASK_AMT_x                   8747
IS_SUCCESSFUL_x                2
STATUS_y                       2
ASK_AMT_y                   8747
                            ... 
INCOME_AMT_25000-99999         2
INCOME_AMT_50M+                2
INCOME_AMT_5M-10M              2
SPECIAL_CONSIDERATIONS_N       2
SPECIAL_CONSIDERATIONS_Y       2
Length: 74, dtype: int64

In [21]:
df_dropped_columns['USE_CASE_Other'].value_counts()

0    34296
1        3
Name: USE_CASE_Other, dtype: int64

In [22]:
# Split our preprocessed data into our features and target arrays
# Dropping more columns to optimize model
y = df_dropped_columns['IS_SUCCESSFUL_x'].values
X = df_dropped_columns.drop(['IS_SUCCESSFUL_x', 'IS_SUCCESSFUL_y', 'ASK_AMT_y', 'SPECIAL_CONSIDERATIONS_N'], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [23]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [24]:
# Increasing the number of hidden layers and the number of neurons in each layer
# Use different activation functions for some hidden layers

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 500
hidden_nodes_layer2 = 450
hidden_nodes_layer3 = 300
hidden_nodes_layer4 = 200
hidden_nodes_layer5 = 100
hidden_nodes_layer6 = 100
hidden_nodes_layer7 = 100

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="tanh"))

# Sixth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="sigmoid"))

# Seventh hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer7, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               35500     
                                                                 
 dense_1 (Dense)             (None, 450)               225450    
                                                                 
 dense_2 (Dense)             (None, 300)               135300    
                                                                 
 dense_3 (Dense)             (None, 200)               60200     
                                                                 
 dense_4 (Dense)             (None, 100)               20100     
                                                                 
 dense_5 (Dense)             (None, 100)               10100     
                                                                 
 dense_6 (Dense)             (None, 100)               1

In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
# Change the number of epochs to the training regimen
fit_model = nn.fit(X_train_scaled,y_train,epochs=25)

Epoch 1/25
804/804 [==============================] - 11s 10ms/step - loss: 0.5779 - accuracy: 0.7172
Epoch 2/25
804/804 [==============================] - 6s 7ms/step - loss: 0.5585 - accuracy: 0.7298
Epoch 3/25
804/804 [==============================] - 5s 6ms/step - loss: 0.5558 - accuracy: 0.7285
Epoch 4/25
804/804 [==============================] - 6s 7ms/step - loss: 0.5507 - accuracy: 0.7301
Epoch 5/25
804/804 [==============================] - 5s 6ms/step - loss: 0.5489 - accuracy: 0.7328
Epoch 6/25
804/804 [==============================] - 5s 7ms/step - loss: 0.5495 - accuracy: 0.7320
Epoch 7/25
804/804 [==============================] - 5s 6ms/step - loss: 0.5497 - accuracy: 0.7309
Epoch 8/25
804/804 [==============================] - 5s 7ms/step - loss: 0.5481 - accuracy: 0.7332
Epoch 9/25
804/804 [==============================] - 5s 6ms/step - loss: 0.5467 - accuracy: 0.7332
Epoch 10/25
804/804 [==============================] - 5s 6ms/step - loss: 0.5464 - accuracy: 0.73

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5552 - accuracy: 0.7278 - 657ms/epoch - 2ms/step
Loss: 0.5551986694335938, Accuracy: 0.7278134226799011


In [28]:
# Export our model to HDF5 file
from tensorflow import keras

nn.save("AlphabetSoupCharity_Optimization.h5")